# EXE Troubleshooting Notes

## Troubleshooting by Arcady

I am working on a Windows 11 computer and using the Powershell console.

## Troubleshooting steps

### September 18, 2025

1. Generated an EXE of the project with Pyinstaller
2. Got **ModuleNotFoundError** on this line in main.py: `import qtmodern.styles`
3. Created a hooks directory within scripts
4. Created hooks\hook-qtmodern.py (file contents in code snippets below)
5. Added qtmodern and PyQt5 to the Poetry venv (PyQt5 was a mistake as the project actually uses PyQt6; I fixed that later)
6. Updated main.spec to collect data files for qtmodern and PyQt5
7. Specified "qtmoderns" as an hidden import in main.spec
8. Removed and added qtmodern to the Poetry venv
10. Asked [a question](https://stackoverflow.com/questions/79767848/exe-created-with-pyinstaller-fails-to-import-qtmodern-as-specified-in-hook) on StackOverflow to get outside input
11. Got the virtual environment’s path and stored it in a variable called $venvpath in Powershell
12. Generated an EXE using main.py rather than main.spec (this makes Pyinstaller overwrite the main.spec file), with command `poetry run pyinstaller main.py --onefile --paths "$venvPath\Lib\site-packages"  --hidden-import=qtmodern  --hidden-import=qtmodern.styles --hidden-import=qtmodern.windows`
13. Got **ModuleNotFoundError** three lines further than before in main.py: `from event_connector import QtEvents, connect_decorated_methods, connect_event`
14. Since event_connector.py was located in scripts with main.py, added an empty __init__.py file to scripts
15. Edited main.spec to add event_connector to hiddenimports and "." to pathex
16. Pyinstaller **would no longer generate an EXE** file at all. Replaced "." in main.spec’s pathex with the full path on my machine
17. Ran `poetry remove PyQt5` and `poetry add PyQt6`
18. There were **traces of PyQt5 remaining**
19. Deleted the venv and recreated it with command `Invoke-Expression (poetry env activate)` (Powershell-specific syntax)
20. Now able to generate an EXE and run it, got the **ModuleNotFoundError** on the qtmodern import again
21. Removed the active folder from main.spec’s pathex after an online search showed me that Pyinstaller doesn’t handle multiple elements in the pathex list well
22. Ran `poetry remove qtmodern` and `poetry add qtmodern`
23. Pyinstaller gave this **error message** while trying to generate an EXE: `Aborting build process due to attempt to collect multiple Qt bindings packages: attempting to run hook for 'PySide6', while hook for 'PyQt6' has already been run!`
24. Added `excludes['PySide6'],` to  main.spec
25. Now able to generate an EXE and run it, got this error: `ImportError: Error importing numpy: you should not try to import numpy from its source directory; please exit the numpy source tree, and relaunch your python interpreter from there.`
26. Added `collect_data_files("numpy")` to datas in main.spec
27. Got a **ValueError** when trying to generate the EXE: `too many values to unpack (expected 2)`
28. Tried to create a function that would flatten the input and put it into two lists and pass that to datas
29. Emptied datas
30. Learned from an online search that there is a known issue when using Numpy 1.27 with 3.12 in a frozen environment
31. Downloaded, installed and added Python 3.11 to the venv
32. When running the EXE generated with Python 3.11, I get a **PyQt error**: `qtpy.QtBindingsNotFoundError: No Qt bindings could be found` from the `import qtmodern.styles` line in main.py
33. Tried to include everything that seemed relevant in hiddenimports (see code snippets below)
34. Created `hooks/hook-qtpy.py` (file contents in code snippets below)

## Code snippets

File created on September 18, step 4

```
#hook-qtmodern.py
from PyInstaller.utils.hooks import collect_data_files

datas = collect_data_files("qtmodern", includes=["**/*.qss"])

hiddenimports = [
    "qtmodern.styles",
    "qtmodern.windo
```

main.spec’s hiddenimports on September 18, step 33

```
hiddenimports=["qtmodern", "qtmodern.styles", "qtmodern.windows", "PyQt6.QtCore", "PyQt6.QtGui", "PyQt6.QtWidgets", "PyQt6.QtPrintSupport", "PyQt6.QtSvg", "PyQt6.QtOpenGL", "event_connector", "numpy"],
```

File created on September 18, step 34

```
#hook-qtpy.py
hiddenimports = [
    "PyQt6",
    "PyQt6.QtCore",
    "PyQt6.QtGui",
    "PyQt6.QtWidgets",
    "PyQt6.QtPrintSuppor",


...``


wsin-hook